In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from scipy.spatial import Voronoi
import vispy as vp


import tyssue

from tyssue.core.objects import Epithelium
from tyssue.geometry import bulk_geometry as bgeom
from tyssue.core.generation import (data_dicts, make_df,
                                    hexa_grid3d, from_3d_voronoi)

from vispy import app, gloo, visuals, scene
from vispy.geometry import MeshData


### Scatter plots of the cell centers

```python
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*grid.T, c='k', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()
```

In [2]:
grid = hexa_grid3d(3, 4, 3)

datasets = from_3d_voronoi(Voronoi(grid))
eptm = Epithelium('start3D', datasets)

gspecs = eptm.set_geom(bgeom)
bgeom.update_all(eptm)
eptm.cell_df.head()

,vol,num_faces,x,z,is_alive,y,area
cell,,,,,,,
0,0.000000,4,-0.125,0.6875,1,0.75,0
1,0.020833,8,1.000,0.5000,1,0.75,0
2,0.000000,4,1.750,0.5000,1,0.75,0
3,0.052083,8,-0.125,0.6875,1,1.50,0
4,0.083333,16,1.000,0.5000,1,1.50,0


In [3]:
bounds = [[-0.1, 2], [0, 2], [0, 10]]

je_out = eptm.cut_out(bounds)

In [4]:
eptm.remove(je_out)

In [8]:
vertices, faces, face_mask = eptm.triangular_mesh(eptm.coords)

canvas = scene.SceneCanvas(keys='interactive', show=True)

grid = canvas.central_widget.add_grid()
view = grid.add_view(0, 1)
#view = canvas.central_widget.add_view()
view.camera =  'turntable'
view.camera.aspect = 1


view.bgcolor = vp.color.Color('#000000')



mesh = vp.scene.visuals.Mesh(vertices=vertices,
                             faces=faces)

wire_pos = vertices[eptm.Nc:].copy()


wire = vp.scene.visuals.Line(pos=wire_pos, 
                             connect=faces[:, :2] - eptm.Nc,
                             color=[0.1, 0.1, 0.3, 0.8],
                             width=1)
ccenters = vp.scene.visuals.Markers(pos=eptm.cell_df[eptm.coords].values,
                                    face_color=[1, 1, 1])

view.add(mesh)
view.add(wire)
view.add(ccenters)
#view.camera.set_range()
canvas.show()

#vp_widget = VispyWidget()
#vp_widget.set_canvas(canvas)



app.run()

0